In [8]:
import matplotlib
#matplotlib.use('pgf')
#matplotlib.rcParams['pgf.preamble'] = [r'\usepackage{url}', ]
%pylab
from pandas import read_csv
import webbrowser
import pandas as pd

plot_points_without_errors = True
color_quantity = "Year"

data = read_csv("alphaplot.csv",skip_blank_lines=True)
data = data[~np.isnan(data["Slope (Salpeter=2.35)"])]

references = data["Reference"]
system = data["System"]
Z = data["Metallicity [Z/H]"]
#cut = (system=="ONC") #(Z<=-0.5) #np.ones(len(data),dtype=np.bool) #(system=="ONC")#*(year <= 2003)
#data = data[cut]
references = data["Reference"]
year = np.array([int(r[:4]) if type(r)==str else -1 for r in references])
slope = -data["Slope (Salpeter=2.35)"]+1
mlow = data["Lower mass (Msun)"]
mhi = data["Upper mass (Msun)"]
mmed = (mlow*mhi)**0.5
slope_error = data["Slope uncertainty"]
slope_upper = -data["Upper limit (1 sigma)"]+1
slope_lower = -data["Lower limit (1 sigma)"]+1

#cut = system
fig, ax = plt.subplots(1,1,figsize=(6,3))

if color_quantity=="Year":
    colors = plt.get_cmap('viridis')((year-year.min()).clip(0,1e100)/(year-year.min()).max())
elif color_quantity=="Metallicity":
    colors = plt.get_cmap('viridis')((Z-Z.min())/(Z.max()-Z.min()))
else:
    colors=np.zeros((len(data),3))

# first plot asymmetric errors where available
i = np.isfinite(slope_lower) * np.isfinite(slope_upper)
ax.errorbar(mmed[i],slope[i],xerr=[mmed[i]-mlow[i],mhi[i]-mmed[i]], yerr=[np.abs(slope-slope_lower)[i],np.abs(slope_upper-slope)[i]], ls='',capsize=0,lw=0.5,c='black',marker=None,ecolor='grey')
ax.scatter(mmed[i],slope[i],c=colors[i],s=2,zorder=10) #xerr=[mmed[i]-mlow[i],mhi[i]-mmed[i]], yerr=[(slope-slope_lower)[i],(slope_upper-slope)[i]], ls='',capsize=0,lw=0.5,c='black',marker='o',markersize=1)

# now symmetric errors
i = np.isfinite(slope_error)# * np.isfinite(slope_upper)
ax.errorbar(mmed[i],slope[i],xerr=[mmed[i]-mlow[i],mhi[i]-mmed[i]], yerr=slope_error[i], ls='',capsize=0,lw=0.5,c='black',marker=None,markersize=0,ecolor='grey')
ax.scatter(mmed[i],slope[i],c=colors[i],s=2,zorder=10)

# and now ones without any errorbars :(
if plot_points_without_errors:
    i = np.isnan(slope_error)
    ax.errorbar(mmed[i],slope[i],xerr=[mmed[i]-mlow[i],mhi[i]-mmed[i]], ls='',capsize=0,lw=0.5,color='black',marker='o',markersize=0,ecolor='grey')
    ax.scatter(mmed[i],slope[i],c=colors[i],s=2,zorder=10)


#colorbar
if color_quantity=="Year":
    import matplotlib.ticker as ticker
    def fmt(x, pos):
        a, b = '{:.2e}'.format(x).split('e')
        b = int(b)
        return r'%d'%int(x)

    sc = ax.scatter(np.zeros(len(mmed)+1),-100*np.ones(len(mmed)+1),c=np.int_(list(year)+[year.min()]),s=2)
    plt.colorbar(sc,format=ticker.FuncFormatter(fmt),pad=0,label="Year")
elif color_quantity=="Metallicity":
    sc = ax.scatter(np.zeros(len(mmed)),-100*np.ones(len(mmed)),c=Z,s=2)
    plt.colorbar(sc,label="Z",pad=0)


if len(np.unique(data["System"].values))==1: 
    ax.set_title(data["System"][0])

# plot dummy points for URLS
# df = pd.DataFrame({'x': mmed,
#                    'y': slope,
#                    'link': [r"https://ui.adsabs.harvard.edu/abs/"+r for r in references]})

# def on_pick(event):
#     url = df.link.iloc[event.ind[0]]
#     webbrowser.open_new_tab(url)

# ax.scatter(x=df.x, y=df.y,picker=5,s=1,alpha=0)

urls = [r"https://ui.adsabs.harvard.edu/abs/"+r for r in references]
# import matplotlib.patches as patches
sc = ax.scatter(mmed,slope,alpha=1e-6,zorder=10000)
sc.set_urls(urls)
for x,y,u in zip(mmed,slope,urls):
    txt = ax.text(x, y, "o", url=u, alpha=1e-6, fontsize=3,bbox=dict(boxstyle='circle',url=u,alpha=1e-6),ha='center',va='center',zorder=10000) #bbox = dict(color='w', alpha=0.01, url=u)
    #ax.add_patch(patches.Rectangle((x,y),0.1,0.1,url=u))#, url=u, alpha=1e-6, fontsize=3,bbox=dict(boxstyle='circle',url=u,alpha=1e-6),ha='center',va='center',zorder=10000)

ax.plot([0.003,300],[0,0],ls='dashed',color='black',zorder=-1000,lw=0.5)
ax.text(10,0.05,'Peak/Plateau',color='black',fontsize=8)
ax.plot([0.003,300],[-1.35,-1.35],ls='dotted',color='black',zorder=-1000,lw=0.5)
ax.text(0.01,-1.3,"Salpeter (-1.35)",color='black',fontsize=8)
ax.plot([0.003,300],[-1.,-1.],ls='dotted',color='black',zorder=-1000,lw=0.5)
ax.plot([0.003,300],[-1,-1.],ls='dotted',color='black',zorder=-1000,lw=0.5)

#ax.plot([0.2,0.4],[-1,-1],color="black",lw=0.5)
#ax.plot([0.2,0.4],[1,1],color="black",lw=0.5)
ax.set(xscale='log',xlabel=r"$M_{\rm ZAMS}$ ($M_\odot$)", ylabel=r"IMF Slope",xlim=[0.003,300],ylim=[-3,3.5])

#fig.canvas.mpl_connect('pick_event', on_pick)
plt.savefig("IMF_AlphaPlot.pdf",bbox_inches='tight')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [12]:
len(mmed)

85

In [21]:
data

,System,Reference,Lower mass (Msun),Upper mass (Msun),Slope (Salpeter=2.35),Slope uncertainty,Lower limit (1 sigma),Upper limit (1 sigma),Metallicity [Z/H],Models binarity?,Notes
0,$\Theta$ Ori,2001ApJ...556..830B,0.013,0.2,0.80,0.40,NaN,NaN,0.0,False,NaN
1,30 Dor,1999A&A...347..532S,3.000,120.0,2.37,0.08,NaN,NaN,-0.5,False,r>3.6pc
2,30 Dor,1999A&A...347..532S,2.800,120.0,2.17,0.05,NaN,NaN,-0.5,False,1.1pc < r < 4.5
3,30 Dor,2000ApJ...533..203S,1.350,2.1,1.27,0.08,NaN,NaN,-0.5,False,NaN
4,30 Dor,2000ApJ...533..203S,2.100,6.5,2.28,0.05,NaN,NaN,-0.5,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
107,Westerlund 1,2011MNRAS.412.2469G,2.500,25.0,2.44,NaN,2.36,2.64,0.0,False,NaN
108,Westerlund 1,2013AJ....145...46L,5.000,100.0,1.80,0.10,NaN,NaN,0.0,False,NaN
109,Westerlund 1,2002ApJ...573..366M,0.600,7.0,2.00,0.10,NaN,NaN,0.0,False,NaN
110,Westerlund 1,2002ApJ...573..366M,0.600,7.0,1.80,0.20,NaN,NaN,0.0,False,NaN
